# AdventureWorks - Hard

In [1]:
import $ivy.`org.apache.spark::spark-sql:3.4.0`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

val spark = {
    NotebookSparkSession.builder()
    .progress(false)
    .appName("app14-3")
    // .master("spark://192.168.31.31:7077")
    .master("local[*]")
    .config("spark.sql.warehouse.dir", 
            "hdfs://192.168.31.31:9000/user/hive/warehouse") 
    .config("spark.cores.max", "4") 
    .config("spark.executor.instances", "1") 
    .config("spark.executor.cores", "2") 
    .config("spark.executor.memory", "10g") 
    .config("spark.shuffle.service.enabled", "false") 
    .config("spark.dynamicAllocation.enabled", "false") 
    .config("spark.sql.catalogImplementation", "hive")
    .config("spark.sql.repl.eagerEval.enabled", "true")
    .config("spark.driver.allowMultipleContexts", "true")
    .getOrCreate()
}

Loading spark-stubs, spark-hive
Adding Hive conf dir /opt/hive/conf to classpath
Creating SparkSession


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


Spark UI

import $ivy.$                                  


import org.apache.log4j.{Level, Logger}

import org.apache.spark._

import org.apache.spark.sql._

import org.apache.spark.sql.types._

import org.apache.spark.sql.functions._

import org.apache.spark.sql.expressions.Window


spark: SparkSession = org.apache.spark.sql.SparkSession@25a4a4ea

In [2]:
import spark.implicits._
def sc = spark.sparkContext
val hiveCxt = new org.apache.spark.sql.hive.HiveContext(sc)

import spark.implicits._

defined function sc
hiveCxt: sql.hive.HiveContext = org.apache.spark.sql.hive.HiveContext@341b663b

In [3]:
// Credit to Aivean
implicit class RichDF(val ds:DataFrame) {
    def showHTML(limit: Int = 50, truncate: Int = 100) = {
        import xml.Utility.escape
        val data = ds.take(limit)
        val header = ds.schema.fieldNames.toSeq        
        val rows: Seq[Seq[String]] = data.map { row =>
          row.toSeq.map {cell =>
            val str = cell match {
              case null => "null"
              case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
              case array: Array[_] => array.mkString("[", ", ", "]")
              case seq: Seq[_] => seq.mkString("[", ", ", "]")
              case _ => cell.toString
            }
            if (truncate > 0 && str.length > truncate) {
              // do not show ellipses for strings shorter than 4 characters.
              if (truncate < 4) str.substring(0, truncate)
              else str.substring(0, truncate - 3) + "..."
            } else {
              str
            }
          }: Seq[String]
        }
    publish.html(s""" <table>
                <tr>
                 ${header.map(h => s"<th>${escape(h)}</th>").mkString}
                </tr>
                ${rows.map {row =>
                  s"<tr>${row.map{c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
                }.mkString}
            </table>
        """)
    }
}

defined class RichDF

In [4]:
val cust_aw = hiveCxt.table("sqlzoo.CustomerAW")
val cust_addr = hiveCxt.table("sqlzoo.CustomerAddress")
val addr = hiveCxt.table("sqlzoo.Address")
val product = hiveCxt.table("sqlzoo.Product")
val order_det = hiveCxt.table("sqlzoo.SalesOrderDetail")
val order_head = hiveCxt.table("sqlzoo.SalesOrderHeader")
val prod_model = hiveCxt.table("sqlzoo.ProductModel")
val prod_model_prod = hiveCxt.table("sqlzoo.ProductModelProductDescription")
val prod_desc = hiveCxt.table("sqlzoo.ProductDescription")
val prod_cat = hiveCxt.table("sqlzoo.ProductCategory")

cust_aw: DataFrame = [customerid: int, namestyle: string ... 11 more fields]
cust_addr: DataFrame = [customerid: int, addressid: int ... 1 more field]
addr: DataFrame = [addressid: int, addressline1: string ... 5 more fields]
product: DataFrame = [productid: int, name: string ... 12 more fields]
order_det: DataFrame = [salesorderid: int, salesorderdetailid: int ... 4 more fields]
order_head: DataFrame = [salesorderid: int, revisionnumber: int ... 17 more fields]
prod_model: DataFrame = [productmodelid: int, name: string ... 1 more field]
prod_model_prod: DataFrame = [productmodelid: int, productdescriptionid: int ... 1 more field]
prod_desc: DataFrame = [productdescriptionid: int, description: string]
prod_cat: DataFrame = [productcategoryid: int, parentproductcategoryid: int ... 1 more field]

## 11.
**For every customer with a 'Main Office' in Dallas show AddressLine1 of the 'Main Office' and AddressLine1 of the 'Shipping' address - if there is no shipping address leave it blank. Use one row per customer.**

In [5]:
(cust_aw.join(cust_addr, "CustomerID")
 .join(cust_aw
       .join(cust_addr.filter(cust_addr("AddressType")==="Main Office"),
             "CustomerID")
       .join(addr.filter(addr("City")==="Dallas"), "AddressID") 
       .select("CustomerID")
       .distinct(), 
       "CustomerID")
 .join(addr, "AddressID")
 .select("CompanyName", "AddressType", "AddressLine1")
 .distinct()
 .groupBy("CompanyName")
 .pivot("AddressType")
 .agg(first("AddressLine1"))
 .na.fill("")
 .showHTML())

CompanyName,Main Office,Shipping
Elite Bikes,Po Box 8259024,9178 Jumping St.
Rental Bikes,"99828 Routh Street, Suite 825",
Third Bike Store,2500 North Stemmons Freeway,
Town Industries,P.O. Box 6256916,
Unsurpassed Bikes,Po Box 8035996,


## 12.
**For each order show the SalesOrderID and SubTotal calculated three ways:**

- **A) From the SalesOrderHeader**
- **B) Sum of OrderQty*UnitPrice**
- **C) Sum of OrderQty*ListPrice**

In [6]:
(order_head.select("SalesOrderID", "SubTotal")
 .join(order_det
       .withColumn("SubTotal", col("OrderQty") * col("UnitPrice") * 
                   (lit(1)-col("UnitPriceDiscount")))
       .groupBy("SalesOrderID")
       .agg(sum("SubTotal").alias("UnitPriceSum"))
       .withColumn("UnitPriceSum", round($"UnitPriceSum", 2)),
      "SalesOrderID")
 .join(order_det
       .join(product, "ProductID")
       .withColumn("SubTotal", col("OrderQty") * col("ListPrice"))
       .groupBy("SalesOrderID")
       .agg(sum("SubTotal").alias("ListPriceSum"))
       .withColumn("ListPriceSum", round($"ListPriceSum", 2)),
      "SalesOrderID")
 .orderBy("SalesOrderID")
 .showHTML())

SalesOrderID,SubTotal,UnitPriceSum,ListPriceSum
71774,880.35,713.8,1189.66
71776,78.81,63.9,106.5
71780,38418.69,29922.81,56651.56
71782,39785.33,33319.68,55533.31
71783,83858.43,65682.74,121625.43
71784,108561.83,89868.88,151932.58
71796,57634.63,47848.02,79746.71
71797,78029.69,65122.79,108986.4
71815,1141.58,926.91,1544.86
71816,3398.17,2847.37,4745.68


## 13.
**Show the best selling item by value.**

In [7]:
(order_det.join(product, "ProductID")
 .withColumn("SubTotal", col("OrderQty") * col("UnitPrice"))
 .groupBy("ProductID", "Name")
 .sum("SubTotal")
 .withColumn("sum(SubTotal)", round($"sum(SubTotal)", 2))
 .orderBy(col("Sum(SubTotal)").desc)
 .showHTML())

ProductID,Name,sum(SubTotal)
969,"Touring-1000 Blue, 60",37191.44
783,"Mountain-200 Black, 42",37178.73
782,"Mountain-200 Black, 38",35801.74
976,"Road-350-W Yellow, 48",33509.58
957,"Touring-1000 Yellow, 60",23745.32
967,"Touring-1000 Blue, 50",22887.04
780,"Mountain-200 Silver, 42",20879.85
973,"Road-350-W Yellow, 40",20411.8
784,"Mountain-200 Black, 46",19277.86
781,"Mountain-200 Silver, 46",18095.87


## 14.
**Show how many orders are in the following ranges (in $):**

```
    RANGE      Num Orders      Total Value
    0-  99
  100- 999
 1000-9999
10000-
```

In [8]:
(order_head
 .withColumn("RANGE", 
             when(col("SubTotal").between(0, 99.99), "    0-  99")
             .when(col("SubTotal").between(100, 999.99), "  100- 999")
             .when(col("SubTotal").between(1000, 9999.99), " 1000-9999")
             .otherwise("10000-    "))
 .groupBy("RANGE")
 .agg(count("SubTotal"), sum("SubTotal"))
 .withColumn("sum(SubTotal)", round($"sum(SubTotal)", 2))
 .orderBy("RANGE")
 .showHTML())

RANGE,count(SubTotal),sum(SubTotal)
0- 99,3,158.66
100- 999,5,2386.21
1000-9999,10,27561.43
10000-,14,835326.81


## 15.
**Identify the three most important cities. Show the break down of top level product category against city.**

In [9]:
(addr
 .join(addr
       .join(order_head, (addr("AddressID")===order_head("ShipToAddressID")))
       .groupBy("City")
       .sum("SubTotal")
       .orderBy(col("sum(SubTotal)").desc)
       .limit(3),
      "City")
 .join(order_head, (addr("AddressID")===order_head("ShipToAddressID")))
 .join(order_det, "SalesOrderID")
 .join(product, "ProductID")
 .join(prod_cat.withColumnRenamed("name", "catg_name"), 
       "ProductCategoryID")
 .withColumn("amount", col("OrderQty") * col("UnitPrice"))
 .groupBy("City", "catg_name")
 .sum("amount")
 .withColumn("sum(amount)", round($"sum(amount)", 2))
 .orderBy("City", "catg_name")
 .showHTML())

City,catg_name,sum(amount)
London,Bottom Brackets,388.73
London,Brakes,255.6
London,Chains,36.42
London,Cranksets,1773.81
London,Derailleurs,638.85
London,Gloves,88.14
London,Handlebars,292.63
London,Helmets,20.99
London,Mountain Bikes,50881.99
London,Mountain Frames,24018.8


In [10]:
spark.stop()